In [5]:
# Install required package
# %pip install langchain-openai
# 
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from typing import Optional
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model

class Vehicle(BaseModel):
    name: Optional[str] = Field(description="The name of the vehicle")
    year: Optional[int] = Field(description="The year of the vehicle")
    color: Optional[str] = Field(description="The color of the vehicle")

class data(BaseModel):
    motor_vehicles: list[Vehicle] 


def get_llm():
    llm = init_chat_model(
        model="gemini-1.5-flash", 
        temperature=0, 
        model_provider="google_genai", 
        api_key=os.getenv("GOOGLE_API_KEY"),
        max_tokens=500  # Limit tokens for faster response
    )
    return llm.with_structured_output(schema=data)



In [7]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are an extraction algorithm. 
     Extract ALL vehicles/cars mentioned in the text.
     For each vehicle, extract the name/brand, year, and color if available.
     If no vehicles are found, return an empty list.
     Make sure to capture ALL vehicles mentioned, not just the first one."""),
    ("human", "{text}")
])

In [8]:
print("Setting up LLM...")
llm = get_llm()

text = [
    "The kia car is a red car",
    "The toyoyo car is a blue car and purchased in 2025",
]

# Join the list items into a single string for better processing
combined_text = ". ".join(text)
print(f"Processing text: {combined_text}")

prompt = prompt_template.invoke({"text": combined_text})
print("Calling LLM...")

# Add timeout handling for faster feedback
try:
    result = llm.invoke(prompt)
    print("Success! Result:")
    print(result)
except Exception as e:
    print(f"Error: {e}")
    print("Try running again or check your API key/internet connection")

Setting up LLM...
Processing text: The kia car is a red car. The toyoyo car is a blue car and purchased in 2025
Calling LLM...
Success! Result:
motor_vehicles=[Vehicle(name='kia', year=2023, color='red'), Vehicle(name='toyoyo', year=2025, color='blue')]
